In [24]:
import geopandas as gpd
import rasterio
from rasterio.transform import Affine
import numpy as np
from pyproj import Proj, transform
import pandas as pd
from scipy.spatial.distance import cdist

# 读取点矢量文件
def read_point_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    return gdf

def get_elevation_values(gdf, raster_path):
    with rasterio.open(raster_path) as src:
        # 获取栅格的地理坐标系信息
        raster_crs = src.crs

        # 将点数据的坐标系转换为与栅格相同的坐标系
        gdf = gdf.to_crs(raster_crs)

        # 计算每个点在栅格上的像素位置
        rows, cols = rasterio.transform.rowcol(src.transform, gdf.geometry.x.values, gdf.geometry.y.values)

        # Convert rows and cols to numpy arrays
        rows = np.array(rows)
        cols = np.array(cols)

        # 将行和列转换为整数，并确保它们位于有效范围内
        rows = np.clip(rows.astype(int), 0, src.height - 1)
        cols = np.clip(cols.astype(int), 0, src.width - 1)

        # 根据行、列索引读取对应像素的高程值
        elevations = src.read(1)[rows, cols]

    gdf['elevation'] = elevations
    return gdf

# 将经纬度转换为平面坐标（根据实际投影需求选择合适的proj4字符串）
def latlon_to_xy(gdf, proj4_string):
    in_proj = Proj(init='epsg:4326')  # WGS84
    out_proj = Proj(proj4_string)
    
    x, y = transform(in_proj, out_proj, gdf.geometry.x, gdf.geometry.y)
    
    gdf['x'] = x
    gdf['y'] = y
    return gdf

# 定义三维高斯函数
def gaussian_3d(point_gdf, sigma=1.0):
    # 提取xyz坐标并重塑为二维数组，每一列分别对应x、y和z坐标
    xyz = point_gdf[['x', 'y', 'elevation']].values.reshape(-1, 3)
    
    # 计算每对点之间的欧氏距离
    distance_matrix = cdist(xyz, xyz, metric='euclidean')
    
    # 应用高斯核函数计算权重矩阵
    weights = np.exp(-distance_matrix / (2 * sigma**2))
    
    return weights

# 构建并输出权重矩阵
def build_and_output_weight_matrix(point_gdf, sigma=1.0):
    # 计算权重矩阵
    weights = gaussian_3d(point_gdf, sigma)

    # 创建DataFrame存储权重矩阵
    df_weights = pd.DataFrame(weights, index=point_gdf.index, columns=point_gdf.index)
    
    # 输出到CSV文件
    df_weights.to_csv('D:/大创/weight_matrix.csv')

# 主程序入口
if __name__ == "__main__":
    shapefile_path = "D:/大创/可燃火点.shp"
    raster_path = "D:/大创/data/云南省.tif"
    proj4_string = "+proj=utm +zone=48 +datum=WGS84 +units=m +no_defs"  # 示例proj4字符串，请替换为实际投影参数

    # 读取点矢量文件
    gdf = read_point_shapefile(shapefile_path)

    # 获取点的高程值
    gdf_with_elev = get_elevation_values(gdf, raster_path)

    # 转换坐标系统
    gdf_projected = latlon_to_xy(gdf_with_elev, proj4_string)

    # 构建并输出权重矩阵
    build_and_output_weight_matrix(gdf_projected)

d:\1\Anaconda3\Anaconda3\envs\myenv\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\34802\AppData\Local\Temp\ipykernel_26052\948999536.py:45: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = transform(in_proj, out_proj, gdf.geometry.x, gdf.geometry.y)
